# Model Server

Please run the following installs once:

    !pip install -U kfserving==0.2.0 lightgbm 
    !pip install -U azure
    !pip uninstall -y mlrun
    !pip install git+https://github.com/mlrun/mlrun.git@development

If you have never run nuclio functions in your notebooks, please uncomment and run the following:

    !pip install nuclio-jupyter

## nuclio code section

In [1]:
# nuclio: ignore
import nuclio

Install the following packages available to the function:

In [2]:
%%nuclio cmd -c
pip install git+https://github.com/mlrun/mlrun.git@development
pip install kfserving==0.2.0 --upgrade
pip install azure
pip install numpy
pip install lightgbm
pip install joblib

In [3]:
import kfserving
import os
import numpy as np
import joblib
from typing import List
import lightgbm as lgb

In [4]:
TARGET_PATH = '/User/mlrun/lightgbm'
MODEL_NAME = 'lightgbm.model.pkl'

In [5]:
class LGBoostModel(kfserving.KFModel):
    def __init__(self, name: str, model_dir: str, booster: lgb.LGBMClassifier = None):
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if booster:
            self.ready = True

    def load(self):
        """Load model from KubeFlow storage.
        """
        model_file = os.path.join(
            kfserving.Storage.download(self.model_dir), MODEL_NAME)

        self.classifier = joblib.load(model_file)

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample.
        
        :param body : A list of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        try:
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.classifier.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")

In [6]:
# nuclio: end-code

____

### deploy

In [7]:
from mlrun import new_model_server, mount_v3io

In [8]:
fn = new_model_server('lgbm_inference_model', 
                      models={'lgbm_joblib': TARGET_PATH}, 
                      model_class='LGBoostModel').apply(mount_v3io())

In [9]:
fn.spec.no_cache=True

In [10]:
#fn.verbose=True
addr = fn.deploy(project='github-demos')

[mlrun] 2020-01-09 12:01:14,772 deploy started
[nuclio] 2020-01-09 12:03:44,077 (info) Build complete
[nuclio] 2020-01-09 12:03:51,163 done updating lgbm-inference-model, function address: 3.20.71.254:30906


### Test data for events

In [11]:
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
import json

import requests

In [12]:
# Grab some data from the test set
features = pq.read_table(os.path.join(TARGET_PATH, 'xtest.parquet')).to_pandas().iloc[:3, :]
labels = pq.read_table(os.path.join(TARGET_PATH, 'ytest.parquet')).to_pandas().iloc[:3, :]

In [13]:
event = {"instances": features.values.tolist()}

In [14]:
resp = requests.post(addr + '/lgbm_joblib/predict', json=event)
json.loads(resp.content)

[1.0, 1.0, 1.0]

### Test of Estimated Model Object 

Here we simply grab the estimated model file created on the [kubeflow pipeline](kubeflow%20pipeline.ipynb), load it, and run a test matrix through it.

In [15]:
import joblib
model_file = os.path.join(TARGET_PATH, MODEL_NAME)
lgbm_model = joblib.load(open(model_file, 'rb'))

In [16]:
testvals = np.asarray(features.values)

In [17]:
lgbm_model.predict(testvals)

array([1., 1., 1.])